In [ ]:
import neonutilities as nu
import pandas as pd 
from neonutilities.aop_download import validate_dpid,validate_site_format,validate_neon_site
from neonutilities.helper_mods.api_helpers import get_api

## Custom function to list available dates for a given data product and site

In [ ]:
def nu_list_available_dates(dpid, site):
    """
		NOTE: This is an internal hack of the original function to return a dataframe instead of printing
        
        nu_list_available_dates displays the available releases and dates for a given product and site
        --------
         Inputs:
             dpid: the data product code (eg. 'DP3.30015.001' - CHM)
             site: the 4-digit NEON site code (eg. 'JORN')
        --------
        Returns:
        prints the Release Tag (or PROVISIONAL) and the corresponding available dates (YYYY-MM) for each tag
    --------
        Usage:
        --------
        >>> list_available_dates('DP3.30015.001','JORN')
        RELEASE-2025 Available Dates: 2017-08, 2018-08, 2019-08, 2021-08, 2022-09

        >>> list_available_dates('DP3.30015.001','HOPB')
        PROVISIONAL Available Dates: 2024-09
        RELEASE-2025 Available Dates: 2016-08, 2017-08, 2019-08, 2022-08

        >>> list_available_dates('DP1.10098.001','HOPB')
        ValueError: There are no data available for the data product DP1.10098.001 at the site HOPB.
    """
    product_url = "https://data.neonscience.org/api/v0/products/" + dpid
    response = get_api(api_url=product_url)  # add input for token?

    # raise value error and print message if dpid isn't formatted as expected
    validate_dpid(dpid)

    # raise value error and print message if site is not a 4-letter character
    site = site.upper()  # make site upper case (if it's not already)
    validate_site_format(site)

    # raise value error and print message if site is not a valid NEON site
    validate_neon_site(site)

    # check if product is active
    if response.json()["data"]["productStatus"] != "ACTIVE":
        raise ValueError(
            f"NEON {dpid} is not an active data product. See https://data.neonscience.org/data-products/{dpid} for more details."
        )

    # get available releases & months:
    for i in range(len(response.json()["data"]["siteCodes"])):
        if site in response.json()["data"]["siteCodes"][i]["siteCode"]:
            available_releases = response.json()["data"]["siteCodes"][i][
                "availableReleases"
            ]

    # display available release tags (including provisional) and dates for each tag
    try:
        availables_list = []
        for entry in available_releases:
            release = entry["release"]
            available_months_str = ", ".join(entry["availableMonths"])
            available_months = [x.strip() for x in available_months_str.split(',')]
            for available_month in available_months:
                availables_list.append({'status':release,'date':available_month})
        available_df = pd.DataFrame(availables_list)
        return(available_df)
    except UnboundLocalError:
        # if the available_releases variable doesn't exist, this error will show up:
        # UnboundLocalError: local variable 'available_releases' referenced before assignment
        raise ValueError(
            f"There are no NEON data available for the data product {dpid} at the site {site}."
        )

In [2]:
site_names = ['DELA','LENO','TALL','BONA','DEJU','HEAL','SRER','SJER','SOAP',
              'TEAK','CPER','NIWO','RMNP','DSNY','OSBS','JERC','PUUM','KONZ',
              'UKFS','SERC','HARV','UNDE','BART','JORN','DCFS','NOGP','WOOD',
              'GUAN','LAJA','GRSM','ORNL','CLBJ','MOAB','ONAQ','BLAN','MLBS',
              'SCBI','ABBY','WREF','STEI','TREE','YELL']

In [ ]:
nu.by_tile_aop(dpid="DP3.30003.001", site="SJER", year="2024", 
          easting=list(trees_sb_latest.adjEasting), 
          northing=list(trees_sb_latest.adjNorthing),
          include_provisional=True,
          savepath=os.getcwd())

## The point cloud data

In [8]:
dp_id = 'DP1.30003.001'  #point clouds 

site_name = 'UNDE'
prod_dates = nu_list_available_dates(dpid=dp_id,site=site_name)
prod_dates


dl_worked = []
dl_failed = []

if prod_dates is None:
	print(f"No point cloud data for {site_name}")
	# continue

for chm_date in prod_dates['date'][0:1]:

	yr_i = chm_date.split('-')[0]
	print(f"starting {yr_i}")

	try:
		nu.by_file_aop(dpid = dp_id,
		site=site_name,
		year=yr_i,
		include_provisional=True,
		check_size=False,
		savepath = '/data/chloris/NEON/')

		dl_worked.append(f"{site_name} - {chm_date}")	
		pd.DataFrame(dl_worked).to_csv('/data/chloris/NEON/dl_succeeded.csv')
		print(f"Downloaded all files for {site_name} - {chm_date}")
	except Exception as e:
		print(f"Failed for site {chm_date}: {e}")
		dl_failed.append(chm_date)
		pd.DataFrame(dl_failed).to_csv('/data/chloris/NEON/dl_failed.csv')

In [9]:
dp_id = 'DP1.30003.001'  #point clouds 

site_name = 'TALL'
prod_dates = nu_list_available_dates(dpid=dp_id,site=site_name)
prod_dates


dl_worked = []
dl_failed = []

if prod_dates is None:
	print(f"No point cloud data for {site_name}")
	# continue

for chm_date in prod_dates['date'][0:1]:

	yr_i = chm_date.split('-')[0]
	print(f"starting {yr_i}")

	try:
		nu.by_file_aop(dpid = dp_id,
		site=site_name,
		year=yr_i,
		include_provisional=True,
		check_size=False,
		savepath = '/data/chloris/NEON/')

		dl_worked.append(f"{site_name} - {chm_date}")	
		pd.DataFrame(dl_worked).to_csv('/data/chloris/NEON/dl_succeeded.csv')
		print(f"Downloaded all files for {site_name} - {chm_date}")
	except Exception as e:
		print(f"Failed for site {chm_date}: {e}")
		dl_failed.append(chm_date)
		pd.DataFrame(dl_failed).to_csv('/data/chloris/NEON/dl_failed.csv')

## The CHM data 

In [ ]:
dp_id = 'DP3.30015.001'
site_names = ['NOGP','WOOD','GUAN','LAJA','GRSM','ORNL','CLBJ','MOAB','ONAQ','BLAN','MLBS','SCBI','ABBY','WREF','STEI','TREE','YELL']

dl_worked = []
dl_failed = []

for site_name in site_names[1:]:
	prod_dates = nu_list_available_dates(dpid='DP3.30015.001',site=site_name)

	if chm_dates is None:
		print(f"No CHM data for {site_name}")
		continue

	for chm_date in chm_dates['date']:

		yr_i = chm_date.split('-')[0]
		print(f"starting {yr_i}")

		try:
			nu.by_file_aop(dpid = dp_id,
			site=site_name,
			year=yr_i,
			include_provisional=True,
			check_size=False,
			savepath = '/data/chloris/NEON/')

			dl_worked.append(f"{site_name} - {chm_date}")	
			pd.DataFrame(dl_worked).to_csv('/data/chloris/NEON/dl_succeeded.csv')
			print(f"Downloaded all files for {site_name} - {chm_date}")
		except Exception as e:
			print(f"Failed for site {chm_date}: {e}")
			dl_failed.append(chm_date)
			pd.DataFrame(dl_failed).to_csv('/data/chloris/NEON/dl_failed.csv')